# C.7.9 Logistic Regression 모델

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('2014DC2.csv')   
df.shape

(12417, 75)

In [3]:
# 참조 코딩
# Finding the columns which have null values.
df.isna().any()[lambda x: x]

Asset7             True
Asset9             True
B2B_purchase1      True
Cost11             True
Cost12             True
Cost13             True
Cost16             True
Cost17             True
Design1            True
ITAssetC           True
Liability3         True
OutsourcingCost    True
Patent1            True
RNDcost1           True
TAssetC1           True
TAssetC2           True
TAssetC3           True
TAssetC4           True
TAssetC5           True
TAssetC6           True
TAssetC8           True
TradeMark1         True
emp66              True
dtype: bool

In [4]:
# 참조 코딩
def missing(dft):
    print (round((dft.isnull().sum() * 100/ len(dft)),2).sort_values(ascending=False))

missing(df)

RNDcost1           43.67
OutsourcingCost    37.66
Asset9             28.36
B2B_purchase1      26.15
Asset7             12.31
                   ...  
EBizSystem11        0.00
EBizSystem12        0.00
Franchise1          0.00
IndCategory2        0.00
Asset2              0.00
Length: 75, dtype: float64


In [ ]:
### *** Regression 기반의 모형 (Logistic Regresssion, LAR, LASSO 등 포함)은
### *** 카테고리 변수를 모두 dummy 변수로 바꾸어야 하며 그래야 Odds ratio가 제대로 구해짐

In [5]:
# dtype이 category인 변수들인 cols.
cols = ['Area', 'Compensation1', 'Compensation2', 'Compensation3', 'Compensation4', 'EBizSystem2','EBizSystem3',\
        'EBizSystem4','EBizSystem5','EBizSystem6','EBizSystem7','EBizSystem8','EBizSystem9','EBizSystem10',\
        'EBizSystem11','EBizSystem12','Franchise1','IndCategory2','NewEntry3',\
        'Outsourcing1', 'Outsourcing2', 'Outsourcing3', 'Outsourcing4', 'Outsourcing5', 'Outsourcing6',\
        'Outsourcing7','Outsourcing8', 'Outsourcing9', 'Outsourcing10', 'Outsourcing11', 'Outsourcing12',\
        'Outsourcing13','Overseas1','Overseas5','ParentCompany1','SAlliance1','StockMktListing','Subsidiary1',\
        'IndCategory1_encoded']

In [6]:
df[cols].describe()

,Area,Compensation1,Compensation2,Compensation3,Compensation4,EBizSystem2,EBizSystem3,EBizSystem4,EBizSystem5,EBizSystem6,...,Outsourcing11,Outsourcing12,Outsourcing13,Overseas1,Overseas5,ParentCompany1,SAlliance1,StockMktListing,Subsidiary1,IndCategory1_encoded
count,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,...,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000,12417.000000
mean,23.480309,1.202142,1.404123,3.529838,3.337118,0.571636,0.056133,0.038737,0.086253,0.110574,...,0.103004,0.028831,0.046549,1.726423,0.705887,1.768543,1.915197,2.817911,1.597890,5.852541
std,10.229595,0.757194,1.139162,1.661398,1.836346,0.494862,0.230187,0.192976,0.280748,0.313617,...,0.303976,0.167339,0.210679,0.445812,0.455661,0.421781,0.278600,0.515958,0.490344,3.930831
min,11.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,11.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,2.000000,3.000000,1.000000,3.000000
50%,24.000000,1.000000,1.000000,4.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,2.000000,3.000000,2.000000,3.000000
75%,31.000000,1.000000,1.000000,5.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,2.000000,3.000000,2.000000,8.000000
max,39.000000,5.000000,5.000000,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,4.000000,2.000000,17.000000


In [7]:
df1 = df.copy()
cols1 = ['Franchise1','NewEntry3','Overseas1','ParentCompany1','SAlliance1','Subsidiary1']
df1[cols1] = df1[cols1].replace(2,0)

In [8]:
# 참조 코딩
df1['Franchise1'].value_counts(dropna=False)

0    12018
1      399
Name: Franchise1, dtype: int64

In [9]:
cols2 = ['Area','Compensation1','Compensation2','Compensation3','Compensation4',
         'EBizSystem12','IndCategory2','IndCategory1_encoded','StockMktListing']

In [10]:
df2 = pd.get_dummies(df1, columns=cols2)   # cols2에 담긴 변수들의 더미변수를 생성 
                                           # 이 명령은 더미변수를 생성한 원본변수는 제거함에 유의

In [11]:
# 참조 코딩
df2.head()

,Asset2,Asset7,Asset9,B2B_purchase1,Capital1,CapitalRatio1,Cost1,Cost11,Cost12,Cost13,...,IndCategory1_encoded_8.0,IndCategory1_encoded_9.0,IndCategory1_encoded_10.0,IndCategory1_encoded_11.0,IndCategory1_encoded_12.0,IndCategory1_encoded_13.0,IndCategory1_encoded_14.0,IndCategory1_encoded_15.0,IndCategory1_encoded_16.0,IndCategory1_encoded_17.0
0,9.945205,9.474088,6.298949,10.365774,10.410305,0.0,11.588830,6.167516,2.639057,3.218876,...,0,0,0,0,0,0,0,0,0,0
1,11.776105,12.201834,9.355133,11.795771,12.966430,0.0,12.199682,7.108244,2.484907,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,9.295325,9.871119,3.218876,10.485033,9.842144,0.0,11.471541,6.184149,1.098612,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,9.295325,5.793014,NaN,NaN,9.671177,0.0,10.951210,4.969813,2.397895,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,7.577634,3.258097,2.833213,7.896925,8.199189,0.0,8.995537,4.094345,0.693147,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df2.shape

(12417, 202)

In [13]:
list(df2.columns)

['Asset2',
 'Asset7',
 'Asset9',
 'B2B_purchase1',
 'Capital1',
 'CapitalRatio1',
 'Cost1',
 'Cost11',
 'Cost12',
 'Cost13',
 'Cost14',
 'Cost16',
 'Cost17',
 'Cost2',
 'Cost9',
 'Design1',
 'Emp_total',
 'EquityShareCapital',
 'ITAssetC',
 'Liability2',
 'Liability3',
 'NProfitB4Tax',
 'OutsourcingCost',
 'Patent1',
 'RNDcost1',
 'RevProfit2',
 'TAssetC1',
 'TAssetC2',
 'TAssetC3',
 'TAssetC4',
 'TAssetC5',
 'TAssetC6',
 'TAssetC8',
 'TradeMark1',
 'emp6',
 'emp66',
 'EBizSystem2',
 'EBizSystem3',
 'EBizSystem4',
 'EBizSystem5',
 'EBizSystem6',
 'EBizSystem7',
 'EBizSystem8',
 'EBizSystem9',
 'EBizSystem10',
 'EBizSystem11',
 'Franchise1',
 'NewEntry3',
 'Outsourcing1',
 'Outsourcing2',
 'Outsourcing3',
 'Outsourcing4',
 'Outsourcing5',
 'Outsourcing6',
 'Outsourcing7',
 'Outsourcing8',
 'Outsourcing9',
 'Outsourcing10',
 'Outsourcing11',
 'Outsourcing12',
 'Outsourcing13',
 'Overseas1',
 'Overseas5',
 'ParentCompany1',
 'SAlliance1',
 'Subsidiary1',
 'Area_11',
 'Area_21',
 'Area_22'

In [14]:
cols3 = ['Area_39','Compensation1_1.0','Compensation2_1.0','Compensation3_1.0',
         'Compensation4_1.0','EBizSystem12_0','IndCategory2_96',
         'IndCategory1_encoded_0.0','StockMktListing_3']
df3 = df2.drop(cols3, axis=1)
df3.shape

(12417, 193)

In [15]:
df3.to_csv('2014DC2_dummy.csv', index=False)

In [16]:
import pandas as pd
import numpy as np
df = pd.read_csv('2014DC2_dummy.csv')   
df.shape

(12417, 193)

In [18]:
# 참조 코딩
# imputation indicator 없는 imputation

data = df.drop(['EBizSystem2'], axis=1)   # 타겟변수를 제외한 변수만 data 데이터프레임에 저장
target = df['EBizSystem2']                # 타겟변수만 target 데이터프레임에 저장

# 50:50 data partition
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
    data, target, test_size=0.5, random_state=42)   # test_size=0.5임에 주의 

# interval 변수의 null value를 평균(mean)으로 impute 
from sklearn.impute import SimpleImputer
imp= SimpleImputer(strategy = 'mean')  
X_train2= imp.fit_transform(X_train) 
X_test2= imp.fit_transform(X_test)   # X_train과 and X_test 둘 다 imputation 적용해야 함에 유의

print("X_train2 shape:", X_train2.shape) 
print("X_test2 shape:", X_test2.shape) 

X_train2 shape: (6208, 192)
X_test2 shape: (6209, 192)


In [17]:
# imputation indicator 생성을 포함한 imputation

data = df.drop(['EBizSystem2'], axis=1)   # 타겟변수를 제외한 입력변수를 data에 저장
target = df['EBizSystem2']                # 타겟변수만 target에 저장

# 50:50 data partition.
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
    data, target, test_size=0.5, random_state=42)   # test_size=0.5임에 주의 

# interval 변수의 null value를 평균(mean)으로 impute 및 add_indicator 포함 
from sklearn.impute import SimpleImputer
imp= SimpleImputer(strategy = 'mean', add_indicator=True)  
X_train2= imp.fit_transform(X_train) 
X_test2= imp.fit_transform(X_test)   # X_train과 and X_test 둘 다 imputation 적용해야 함에 유의

print("X_train2 shape:", X_train2.shape) 
print("X_test2 shape:", X_test2.shape) 

X_train2 shape: (6208, 214)
X_test2 shape: (6209, 215)


In [18]:
simple_imputer = SimpleImputer(strategy = 'mean', add_indicator=True)
simple_imputer.fit(X_train)
print(simple_imputer.indicator_.features_)

[ 1  2  3  8  9 11 12 15 18 20 22 23 24 26 27 28 29 30 31 32 33 35]


In [19]:
simple_imputer = SimpleImputer(strategy = 'mean', add_indicator=True)
simple_imputer.fit(X_test)
print(simple_imputer.indicator_.features_)

[ 1  2  3  7  8  9 11 12 15 18 20 22 23 24 26 27 28 29 30 31 32 33 35]


In [20]:
df.columns.to_numpy()[7]

'Cost11'

In [21]:
df['Cost11'].isnull().sum()

2

In [22]:
# 참조 코딩
(df['Cost11'].isnull().sum()* 100) / len(df['Cost11'])

0.016106950148989288

In [23]:
df['Cost11'].mean()

4.972271200054787

In [24]:
# Cost11은 수동으로 미리 imputation해서 기존 변수를 대체
df['Cost11']= df['Cost11'].replace(np.NaN, 4.97227) 
df['Cost11'].isnull().sum()

0

In [25]:
# imputation indicator 생성을 포함한 imputation

data = df.drop(['EBizSystem2'], axis=1)   # 타겟변수를 제외한 입력변수를 data 데이터프레임 저장
target = df['EBizSystem2']                # 타겟변수만 target에 저장

# 50:50 data partition.
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
    data, target, test_size=0.5, random_state=42)   # test_size=0.5임에 주의 

# interval 변수의 null value를 평균(mean)으로 impute 및 add_indicator 포함 
from sklearn.impute import SimpleImputer
imp= SimpleImputer(strategy = 'mean', add_indicator=True)  
X_train2= imp.fit_transform(X_train) 
X_test2= imp.fit_transform(X_test)   # X_train과 and X_test 둘 다 imputation 적용해야 함에 유의

print("X_train2 shape:", X_train2.shape) 
print("X_test2 shape:", X_test2.shape) 

X_train2 shape: (6208, 214)
X_test2 shape: (6209, 214)


In [26]:
df.shape

(12417, 193)

In [27]:
df.to_csv('2014DC2_dummy_indicator_friendly.csv', index=False) 

In [ ]:
# 본문 파이썬 팁 코딩 시작

In [2]:
dft = pd.DataFrame({'A':['Good','Bad','Good'], 'B':[1,0,1], 'C':[10,20,30]})
dft

,A,B,C
0,Good,1,10
1,Bad,0,20
2,Good,1,30


In [3]:
dft.columns.to_numpy()[1]

'B'

In [ ]:
# 본문 파이썬 팁 코딩 끝